In [220]:
#!pip install pyspark
#!pip install findspark

In [221]:
import findspark
import builtins

from fontTools.designspaceLib.types import locationInRegion
from pyspark.ml import Pipeline

findspark.init()
from cProfile import label

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, month, dayofmonth, abs,
    avg, sum, min, max, stddev, variance, percentile_approx, when,
    concat, lit, date_format
)
from pyspark.sql.functions import abs as spark_abs
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [222]:
# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Rainfall_Prediction_Project_Enhanced") \
    .getOrCreate()

In [223]:
# Đọc dữ liệu
csv_path = "../data/processed/rainfall_clean.csv"
print("CSV path:", csv_path)
# Đọc dữ liệu lượng mưa
df = (spark.read.option("header", True)
      .csv(csv_path, inferSchema=True))

CSV path: ../data/raw/rainfall.csv


In [224]:
# Đếm số dòng và cột
num_rows = df.count()
num_cols = len(df.columns)
print(f"Dataset có {num_rows} dòng và {num_cols} cột")

Dataset có 10000 dòng và 8 cột


In [225]:
# Xem 10 dòng đầu
df.show(10)

+-------------------+--------+-----------+-------------+------------+--------------+------------+-------------+
|               date|location|rainfall_mm|temperature_c|humidity_pct|wind_speed_kmh|pressure_hpa|rain_category|
+-------------------+--------+-----------+-------------+------------+--------------+------------+-------------+
|2020-01-01 00:00:00|  Danang|     109.66|         27.5|        40.2|          31.9|       997.9|        Heavy|
|2020-01-01 01:00:00|    HCMC|     110.75|         26.1|        80.7|          10.9|      1006.1|        Heavy|
|2020-01-01 02:00:00|     Hue|      23.18|         20.3|        60.3|          22.4|      1004.2|     Moderate|
|2020-01-01 03:00:00|     Hue|     159.38|         22.1|        86.4|           9.3|       997.5|        Heavy|
|2020-01-01 04:00:00|  Danang|      122.6|         24.8|        54.1|          25.9|      1007.4|        Heavy|
|2020-01-01 05:00:00|    HCMC|     131.87|         29.7|        57.1|          30.3|      1001.4|       

In [226]:
# Khảo sát nhanh dữ liệu lượng mưa
df.describe("rainfall_mm").show()

+-------+-----------------+
|summary|      rainfall_mm|
+-------+-----------------+
|  count|            10000|
|   mean|99.11817499999945|
| stddev|57.91234235945852|
|    min|             0.01|
|    max|           199.94|
+-------+-----------------+



In [227]:
# Chuyển cột date về kiểu Date
df = df.withColumn("date", F.to_date("date", "yyyy-MM-dd"))

In [228]:
# Tách năm, tháng, ngày để phân tích thời gian
df = df.withColumn("year", F.year("date")) \
       .withColumn("month", F.month("date")) \
       .withColumn("day", F.dayofmonth("date"))

df.select("date", "year", "month", "day").show(5)

+----------+----+-----+---+
|      date|year|month|day|
+----------+----+-----+---+
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
+----------+----+-----+---+
only showing top 5 rows


In [229]:
# Thêm cột season với 4 mùa
df = df.withColumn(
    "season",
    when(col("month").isin(3, 4, 5), "spring")
    .when(col("month").isin(6, 7, 8), "summer")
    .when(col("month").isin(9, 10, 11), "autumn")
    .otherwise("winter")
)

In [230]:
# Tiền xử lý và ép kiểu an toàn
numeric_cols = [
    "rainfall_mm",
    "temperature_c",
    "humidity_pct",
    "wind_speed_kmh",
    "pressure_hpa"
]

for c in numeric_cols:
    df = df.withColumn(
        c,
        F.when(
            F.col(c).rlike(r'^-?\d+(\.\d+)?$'),
            F.col(c).cast(DoubleType())
        ).otherwise(None)
    )

print(" KIỂM TRA GIÁ TRỊ THIẾU:")
print("-" * 40)

total_rows = df.count()
for column in df.columns:
    # Đếm số dòng có giá trị null
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        null_percentage = (null_count / total_rows) * 100
        print(f"{column}: {null_count} giá trị thiếu ({null_percentage:.2f}%)")

 KIỂM TRA GIÁ TRỊ THIẾU:
----------------------------------------


In [231]:
# Xóa các dòng có giá trị thiếu
df_clean = df.dropna()
print(f"Đã xóa dòng có missing values. Số dòng còn lại: {df_clean.count()}")

Đã xóa dòng có missing values. Số dòng còn lại: 10000


THÊM MỘT SỐ BIẾN MỚI QUAN TRỌNG

In [232]:
# 1.Điểm sương
df_clean = df_clean.withColumn(
    "dew_point",
    col("temperature_c") - ((100 - col("humidity_pct")) / 5)
)

In [233]:
# 2.Tương tác nhiệt độ - độ ẩm
df_clean = df_clean.withColumn(
    "temp_humidity_interaction",
    col("temperature_c") * col("humidity_pct")
)

In [234]:
# 3.Tưởng tác gió - độ ẩm
df_clean = df_clean.withColumn(
    "wind_humidity_effect",
    col("wind_speed_kmh") * (col("humidity_pct") / 100)
)

In [235]:
# 4. Biến mùa địa phương
df_clean = df_clean.withColumn(
    "is_rainy_season",
    when(
        (col("location") == "Hue") & (col("month").isin([9, 10, 11, 12])), 1
    ).when(
        (col("location") == "HCMC") & (col("month").isin([5, 6, 7, 8, 9, 10])), 1
    ).when(
        (col("location") == "Danang") & (col("month").isin([9, 10, 11, 12])), 1
    ).when(
        (col("location") == "Hanoi") & (col("month").isin([5, 6, 7, 8, 9])), 1
    ).otherwise(0)
)

In [236]:
# 5. Chênh lệc áp suất (so với trung bình vùng)
window_spec = Window.partitionBy("location")
df_clean = df_clean.withColumn(
    "avg_pressure_by_location",
    F.avg("pressure_hpa").over(window_spec)
)

df_clean = df_clean.withColumn(
    "pressure_deviation",
    col("pressure_hpa") - col("avg_pressure_by_location")
)

In [237]:
# Các biến số (bao gồm biến mới)
numeric_features = [
    "temperature_c", "humidity_pct", "wind_speed_kmh", "pressure_hpa",
    "dew_point",
    "temp_humidity_interaction",
    "wind_humidity_effect",
    "is_rainy_season",
    "avg_pressure_by_location",
]
label_column = "rainfall_mm"
print(f"Tổng số biến số: {len(numeric_features)}")
print(f"Target: {label_column}")

Tổng số biến số: 9
Target: rainfall_mm


In [238]:
# Mã hóa biến phân loại 'location'
indexer = StringIndexer(inputCol="location", outputCol="location_index")
encoder = OneHotEncoder(inputCol="location_index",
                       outputCol="location_encoded")

In [239]:
# Mã hóa biến 'season'
season_indexer = StringIndexer(inputCol="season", outputCol="season_index")
season_encoder = OneHotEncoder(inputCol="season_index",
                              outputCol="season_encoded")


In [240]:
# Tạo danh sách feature cuối cùng
final_features = numeric_features + ["location_encoded", "season_encoded"]
for i, feat in enumerate(final_features, 1):
    print(f"  {i}. {feat}")

  1. temperature_c
  2. humidity_pct
  3. wind_speed_kmh
  4. pressure_hpa
  5. dew_point
  6. temp_humidity_interaction
  7. wind_humidity_effect
  8. is_rainy_season
  9. avg_pressure_by_location
  10. location_encoded
  11. season_encoded


In [241]:
# Tạo Vector Assembler
assembler = VectorAssembler(
    inputCols=final_features,
    outputCol="features"
)

In [242]:
# Tạo pipeline HOÀN CHỈNH
feature_pipeline = Pipeline(stages=[
    indexer, encoder,
    season_indexer, season_encoder,
    assembler
])

In [243]:
# Fit và transform qua pipeline
df_transformed = feature_pipeline.fit(df_clean).transform(df_clean)

In [244]:
# Chọn chỉ columns cần thiết
df_final = df_transformed.select("features", label_column)

In [245]:
print("\nKẾT QUẢ TRANSFORM CUỐI CÙNG:")
print(f"Số mẫu: {df_final.count():,} mẫu")
print(f"Số feature: {len(df_final.first()['features'])} chiều")
print(f"Các cột: {df_final.columns}")



KẾT QUẢ TRANSFORM CUỐI CÙNG:
Số mẫu: 10,000 mẫu
Số feature: 16 chiều
Các cột: ['features', 'rainfall_mm']


In [246]:
print("\n5 MẪU ĐẦU TIÊN (features đã encoded):")
print("-"*60)
df_final.show(5, truncate=False)


5 MẪU ĐẦU TIÊN (features đã encoded):
------------------------------------------------------------
+---------------------------------------------------------------------------------------------------------+-----------+
|features                                                                                                 |rainfall_mm|
+---------------------------------------------------------------------------------------------------------+-----------+
|(16,[0,1,2,3,4,5,6,8,13],[30.5,51.0,24.0,996.4,20.7,1555.5,12.24,1005.0259659969116,1.0])                |113.24     |
|(16,[0,1,2,3,4,5,6,8,13],[27.5,94.5,10.7,990.4,26.4,2598.75,10.1115,1005.0259659969116,1.0])             |198.74     |
|(16,[0,1,2,3,4,5,6,8,13],[28.0,41.8,28.8,1010.4,16.36,1170.3999999999999,12.0384,1005.0259659969116,1.0])|105.91     |
|(16,[0,1,2,3,4,5,6,8,13],[23.7,82.2,39.1,1001.0,20.14,1948.14,32.14020000000001,1005.0259659969116,1.0]) |72.93      |
|(16,[0,1,2,3,4,5,6,8,13],[24.1,75.9,12.1,1019.0,19.28,1829.

In [247]:
def train_rainfall_model(
    df_final,
    feature_columns,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=42,
    scale=True
):
    print("BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH")
    print(f"Test size: {test_size*100}%")
    print(f"Số features: {len(feature_columns)}")
    print("="*60)

    # Kiểm tra dữ liệu
    print("KIỂM TRA DỮ LIỆU")
    if "features" not in df_final.columns:
        raise ValueError("Thiếu column 'features'")
    if label_column not in df_final.columns:
        raise ValueError(f"Thiếu column '{label_column}'")

    # Chia dữ liệu
    train_df, test_df = df_final.randomSplit([1 - test_size, test_size], seed=seed)
    print(f"  Train: {train_df.count()} mẫu")
    print(f"  Test : {test_df.count()} mẫu")

    # Khởi tạo các stages
    stages = []
    if scale:
        scaler = StandardScaler(
            inputCol="features",
            outputCol="scaled_features",
            withMean=True,
            withStd=True
        )
        stages.append(scaler)
        features_col = "scaled_features"
    else:
        features_col = "features"

    # Khởi tạo thuật toán Linear Regression
    lr = LinearRegression(
        featuresCol=features_col,
        labelCol=label_column,
        solver="normal",
        maxIter=100
    )
    stages.append(lr)

    pipeline = Pipeline(stages=stages)

    # Huấn luyện mô hình
    print("\nHUẤN LUYỆN MÔ HÌNH...")
    pipeline_model = pipeline.fit(train_df)

    # lr là stage cuối cùng trong pipeline
    lr_model = pipeline_model.stages[-1]

    # Dự đoán và đánh giá
    predictions = pipeline_model.transform(test_df)

    print("\nĐÁNH GIÁ TRÊN TẬP TEST")
    evaluators = {
        "RMSE": RegressionEvaluator(labelCol=label_column, metricName="rmse"),
        "MAE": RegressionEvaluator(labelCol=label_column, metricName="mae"),
        "R2": RegressionEvaluator(labelCol=label_column, metricName="r2")
    }

    metrics = {}
    for name, evaluator in evaluators.items():
        value = evaluator.evaluate(predictions)
        metrics[name] = value
        print(f"  {name}: {value:.4f}")

    # Thống kê trên tập train
    print("\nTHỐNG KÊ TRÊN TẬP TRAIN")
    summary = lr_model.summary
    print(f"  R2 (train): {summary.r2:.4f}")
    print(f"  RMSE (train): {summary.rootMeanSquaredError:.4f}")

    # Hiển thị độ quan trọng của features
    if len(feature_columns) <= 15:
        print("\nTỶ LỆ ĐÓNG GÓP CỦA CÁC FEATURES (tuyệt đối):")
        coefficients = lr_model.coefficients.toArray().tolist()
        for feat, coef in zip(feature_columns, coefficients):
            print(f"  {feat[:30]:30} : {coef:10.4f}")

    print("HOÀN THÀNH HUẤN LUYỆN")

    return {
        "pipeline_model": pipeline_model,
        "linear_model": lr_model,
        "predictions": predictions,
        "metrics": metrics,
        "feature_importance": lr_model.coefficients.toArray().tolist() if hasattr(lr_model, 'coefficients') else []
    }


In [248]:

print("BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH VỚI BIẾN MỚI")

# Huấn luyện mô hình lần 1
results_1 = train_rainfall_model(
    df_final=df_final,
    feature_columns=final_features,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=42,
    scale=True
)

# Huấn luyện mô hình lần 2 (với seed khác để kiểm tra tính ổn định)
results_2 = train_rainfall_model(
    df_final=df_final,
    feature_columns=final_features,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=142,
    scale=True
)

BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH VỚI BIẾN MỚI
BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH
Test size: 20.0%
Số features: 11
KIỂM TRA DỮ LIỆU
  Train: 8079 mẫu
  Test : 1921 mẫu

HUẤN LUYỆN MÔ HÌNH...

ĐÁNH GIÁ TRÊN TẬP TEST
  RMSE: 57.8764
  MAE: 50.0150
  R2: -0.0010

THỐNG KÊ TRÊN TẬP TRAIN
  R2 (train): 0.0011
  RMSE (train): 57.8848

TỶ LỆ ĐÓNG GÓP CỦA CÁC FEATURES (tuyệt đối):
  temperature_c                  :     1.2640
  humidity_pct                   :     2.2207
  wind_speed_kmh                 :    -2.8221
  pressure_hpa                   :    -0.7109
  dew_point                      :     2.2329
  temp_humidity_interaction      :    -5.5977
  wind_humidity_effect           :     3.6385
  is_rainy_season                :    -0.4484
  avg_pressure_by_location       :     0.1671
  location_encoded               :    -0.4697
  season_encoded                 :     0.3069

HOÀN THÀNH HUẤN LUYỆN
BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH
Test size: 20.0%
Số features: 11
KIỂM TRA DỮ LIỆU
  Train: 7994 mẫu
  Test : 2006 m

In [249]:
spark.stop()
print("\nĐã đóng Spark Session.")


Đã đóng Spark Session.
